# NLP TO CLASIFY SPAM/NOT SPAM:

# Introducción

El objetivo de este proyecto es desarrollar un sistema de clasificación binaria capaz de distinguir entre mensajes **SPAM** y **NO SPAM** utilizando técnicas de *Natural Language Processing* (NLP). Este tipo de problema es especialmente relevante en aplicaciones reales, donde es fundamental filtrar contenido no deseado de forma automática y robusta.

Para abordar la tarea, se comienza implementando un **modelo NLP básico**, basado en un preprocesado clásico del texto y una representación vectorial mediante **TF-IDF**, junto con un clasificador lineal. Sobre este modelo inicial se realiza un proceso de **optimización de hiperparámetros** con el objetivo de maximizar la métrica oficial de la competición, **Matthews Correlation Coefficient (MCC)**, especialmente adecuada en escenarios con posible desbalanceo entre clases.

Una vez establecido este baseline, el proyecto explora un enfoque más avanzado basado en **transfer learning**, utilizando un modelo de lenguaje preentrenado (**DistilBERT**). En esta segunda fase se reaprovechan representaciones semánticas aprendidas previamente a gran escala, aplicando primero entrenamiento de la cabeza de clasificación y posteriormente **fine-tuning** parcial del modelo para adaptarlo al dominio específico del problema.

De este modo, el proyecto permite comparar un enfoque clásico de NLP con uno moderno basado en *Transformers*, analizando sus diferencias en complejidad, rendimiento y capacidad de generalización.

# Librerías

In [2]:
import os
import numpy as np
import tensorflow as tf
seed = 42
tf.random.set_seed(seed)

# Pandas para dataframes
import pandas as pd
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

# Product para simular GridSearch
from itertools import product

# Modelo básico
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# competition metric for local evaluation
from sklearn.metrics import matthews_corrcoef

# Cargar el dataset

En esta sección cargamos los datos del dataset y creamos los distintos subconjuntos (train, validation, test)

Los datos de entrenamiento y testeo se encuentran en archivos csv. Los leemos usando pandas

Para los datos de entrenamiento (train), vamos a reservar el 20% para validación, así evaluamos el rendimiento del modelo durante el entrenamiento 

In [3]:
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col = "row_id")

# Train / Validation split
X = train["text"].values
y = train["spam_label"].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=seed,
    stratify=y
)

train

,text,spam_label
row_id,,
0,"You are everywhere dirt, on the floor, the windows, even on my shirt. And sometimes when i open my mouth, you are all that comes flowing out. I dr...",0
1,"Subject: \nmon , 24 may 2004 12 : 14 : 06 - 0600\ni am taking the liberty of writing you this\nletter instead of interrupting you\nby phone . plea...",1
2,So the sun is anti sleep medicine.,0
3,Hey are you angry with me. Reply me dr.,0
4,Ü go home liao? Ask dad to pick me up at 6...,0
...,...,...
7092,I stayed at this hotel over the weekend of the Chicago Bears Fan Convention (Feb 27- March 1). The hotel is beautiful. I had a Towers room. I had ...,0
7093,Subject: eis invoices for may\ni just wanted to make all of you aware of the message below from financial\noperations . when you review your rc re...,0
7094,Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free,0


In [4]:
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col = "row_id")

X_test = test["text"].values

test

,text
row_id,
7097,"Subject: king ranch processed volumes at tailgate\nd .\nmary provided this to brian , please check against your numbers asap\n- - - - - - - - - - ..."
7098,"Subject: cialis , viagra , xanax , valium at low price ! no prescription needed !\ndiscount rx is simple , quick , and affordable ! br\noffering m..."
7099,"Subject: september deal inactivation in sitara\ncurrently , prior month deals are inactivated in sitara for portfolio\npurposes on the 10 th of th..."
7100,Subject: jan . 01 sale to texas general land office\nlinda -\ni did not know that this was part of a natural gas / crude oil exchange deal\nand bi...
7101,Subject: 5 th changes @ duke and air liquide\n- - - - - - - - - - - - - - - - - - - - - - forwarded by ami chokshi / corp / enron on 02 / 04 / 200...
...,...
11824,"Subject: contract obligations\ncharlie ,\nhere is a breakout for the volumes that have been paid on . we are off by\napproximately 20000 according..."
11825,"For a hotel rated with four diamonds by AAA, one would think the Hilton Chicago would be almost like staying at a palace with royalty. The only ro..."
11826,"We spend our days waiting for the ideal path to appear in front of us.. But what we forget is.. ""paths are made by walking.. not by waiting.."" Goo..."


## Modelo NLP básico

Como primer enfoque se implementa un modelo de NLP clásico, que sirve como **baseline** para el problema de clasificación SPAM/NO SPAM. Este tipo de modelos se basa en una representación explícita del texto y en algoritmos de clasificación lineales, lo que permite una implementación sencilla, eficiente y fácilmente interpretable.

El pipeline del modelo básico consta de las siguientes etapas:

1. Representación del texto mediante **TF-IDF**, transformando cada mensaje en un vector numérico que refleja la importancia relativa de cada término en el mensaje.
2. Inclusión de **n-grams**, permitiendo capturar no solo palabras individuales, sino también combinaciones frecuentes de palabras.
3. Clasificación mediante **regresión logística**, un modelo lineal adecuado para problemas de clasificación binaria.

Este modelo proporciona un punto de partida sólido sobre el cual evaluar mejoras posteriores.

# Optimización de hiperparámetros del modelo

Con el objetivo de maximizar el rendimiento del clasificador, se llevó a cabo una búsqueda de hiperparámetros (*grid search manual*) sobre el vectorizador TF-IDF y el clasificador lineal. Esto permite identificar la combinación que mejor optimiza la métrica oficial de la competición, **Matthews Correlation Coefficient (MCC)**.

Los hiperparámetros evaluados fueron:

- **C:** parámetro de regularización del clasificador lineal (Logistic Regression). Controla el equilibrio entre ajuste del modelo y complejidad; valores mayores reducen la regularización.

- **ngram_range:** tamaño de los n-grams considerados durante la vectorización. Incluir bigrams o trigrams permite capturar patrones léxicos más ricos que el simple modelo unigram.

- **max_features:** número máximo de características generadas por el vectorizador TF-IDF. Este hiperparámetro controla la dimensionalidad del espacio de representación y puede influir tanto en el rendimiento como en el coste computacional.

Cada combinación se entrenó sobre el conjunto de entrenamiento y se evaluó sobre un conjunto de validación independiente utilizando MCC. Finalmente, se seleccionaron los hiperparámetros con mejor desempeño para entrenar el modelo definitivo.

In [5]:
param_grid = {
    "C": [0.5, 1.0, 2.0, 4.0],
    "ngram_range": [(1,1), (1,2), (1,3)],
    "max_features": [10_000, 20_000, 40_000],
}

results = []

# Grid Search
for C, ngram_range, max_features in product(param_grid["C"], param_grid["ngram_range"], param_grid["max_features"]):
    vectorizer = TfidfVectorizer(
        ngram_range=ngram_range,
        max_features=max_features,
        stop_words="english",
    )

    X_train_vec = vectorizer.fit_transform(X_train)
    X_val_vec   = vectorizer.transform(X_val)

    clf = LogisticRegression(
        max_iter=1000,
        C=C,
        class_weight="balanced"
    )

    clf.fit(X_train_vec, y_train)
    y_val_pred = clf.predict(X_val_vec)
    mcc = matthews_corrcoef(y_val, y_val_pred)

    results.append({
        "C": C,
        "ngram_range": str(ngram_range),
        "max_features": max_features,
        "MCC": mcc
    })

# Convertimos los resultados en DataFrame
results_df = pd.DataFrame(results).sort_values(by="MCC", ascending=False)

# Mostramos la tabla ordenada por MCC
results_df

,C,ngram_range,max_features,MCC
34,4.0,"(1, 3)",20000,0.845357
35,4.0,"(1, 3)",40000,0.839634
31,4.0,"(1, 2)",20000,0.837579
28,4.0,"(1, 1)",20000,0.836414
33,4.0,"(1, 3)",10000,0.835523
24,2.0,"(1, 3)",10000,0.833215
32,4.0,"(1, 2)",40000,0.833215
25,2.0,"(1, 3)",20000,0.833096
21,2.0,"(1, 2)",10000,0.832597
29,4.0,"(1, 1)",40000,0.831846


# Entrenamiento del modelo final con los mejores hiperparámetros

Una vez identificada la combinación óptima de hiperparámetros mediante la búsqueda realizada en la sección anterior, se entrena el modelo definitivo utilizando **el conjunto completo de entrenamiento**. El objetivo de este bloque es construir el clasificador final que se usará posteriormente para generar las predicciones sobre el conjunto de test.

Los hiperparámetros seleccionados fueron:

- **C = 4.0:** menor regularización, permitiendo al clasificador ajustar mejor los patrones.

- **ngram_range = (1, 3):** uso de unigramas, bigramas y trigramas, lo cual permite capturar expresiones y secuencias léxicas relevantes para identificar mensajes de spam.

- **max_features = 20 000:** tamaño del vocabulario TF-IDF que proporciona un buen equilibrio entre representatividad y eficiencia computacional.

Con estos parámetros se entrena el vectorizador TF-IDF y el clasificador lineal (*Logistic Regression*) sobre **todo el dataset** disponible, aprovechando así la máxima cantidad de información antes de generar las predicciones finales.

In [6]:
# Obtenemos los mejores hiperparámetros de la optimización
best_row = results_df.iloc[0]

C_best = float(best_row["C"])
ngram_best = eval(best_row["ngram_range"])
max_feat_best = int(best_row["max_features"])

print("Usando mejores parámetros finales:")
print("C =", C_best, "ngram =", ngram_best, "max_features =", max_feat_best)

# Vectorizer
vectorizer = TfidfVectorizer(
    ngram_range=ngram_best,
    max_features=max_feat_best,
    stop_words="english",
)

# Modelo de clasificación
classifier = LogisticRegression(
    max_iter=1000,
    C=C_best,
    class_weight="balanced"
)

# Fit transform con datos de entrenamiento
X_vec = vectorizer.fit_transform(X)
classifier.fit(X_vec, y)

Usando mejores parámetros finales:
C = 4.0 ngram = (1, 3) max_features = 20000


LogisticRegression(C=4.0, class_weight='balanced', max_iter=1000)

In [7]:
# Test
X_test_vec = vectorizer.transform(X_test)

y_pred = classifier.predict(X_test_vec).astype(int)

## Conclusiones del modelo NLP básico

El modelo NLP básico basado en TF-IDF y regresión logística proporciona un rendimiento sólido en la tarea de clasificación SPAM/NO SPAM, alcanzando valores competitivos de MCC tras la optimización de hiperparámetros.

La búsqueda sistemática de hiperparámetros demostró ser clave para mejorar el rendimiento, evidenciando la importancia de ajustar tanto la representación del texto como la regularización del clasificador. En particular, la inclusión de n-grams de mayor tamaño y un vocabulario TF-IDF adecuado permitió capturar patrones léxicos relevantes asociados al spam.

A pesar de su simplicidad, este enfoque clásico constituye un baseline robusto y eficiente, que sirve como referencia para evaluar modelos más avanzados.

## Submit your predictions in a `submission.csv` file for scoring on the [leaderboard](https://www.kaggle.com/competitions/u-tad-spam-not-spam-2025-edition/leaderboard)
To submit your notebook click on **Submit to competition** and then **Submit**.

In [8]:
# do not modify this code
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred
submission.to_csv('submission.csv',index=False)

In [9]:
submission.head()

,row_id,spam_label
0,7097,0
1,7098,1
2,7099,0
3,7100,0
4,7101,0


## Modelo NLP importado

Como segunda aproximación se emplea un modelo de lenguaje preentrenado basado en la arquitectura **Transformer**, concretamente **DistilBERT**. Este modelo ha sido entrenado previamente sobre grandes volúmenes de texto, lo que le permite aprender representaciones contextuales del lenguaje natural.

A diferencia del modelo básico, este enfoque no se basa en una representación fija del texto, sino que genera embeddings dinámicos que tienen en cuenta el contexto completo de cada palabra dentro del mensaje. Esto resulta especialmente útil en tareas como la detección de spam, donde el significado depende fuertemente del contexto.

El uso de un modelo preentrenado permite aplicar técnicas de **transfer learning**, reduciendo el tiempo de entrenamiento y mejorando la capacidad de generalización.

## Preparación de datos para el modelo importado

Para el modelo basado en Transformers, el texto se prepara utilizando un **tokenizador específico del modelo DistilBERT**, encargado de convertir cada mensaje en una secuencia de tokens compatible con el modelo.

Esta etapa incluye:
- Tokenización del texto.
- Truncado o padding a una longitud máxima fija.

A diferencia del modelo clásico, no se realiza una limpieza del texto, ya que el modelo ha sido preentrenado con texto natural completo y se beneficia de conservar la estructura original del lenguaje.

In [ ]:
!pip install transformers -U

In [11]:
# Modelo para transfer learning y fine tunning
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [13]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

def encode(texts):
    return tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=256,
        return_tensors="tf"
    )

enc_train = encode(X_train)
enc_val   = encode(X_val)

train_ds = tf.data.Dataset.from_tensor_slices((dict(enc_train),y_train)).shuffle(2048, seed=seed).batch(16).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((dict(enc_val),y_val)).batch(32).prefetch(tf.data.AUTOTUNE)

enc_test = encode(test["text"].astype(str).values)
test_ds = tf.data.Dataset.from_tensor_slices(dict(enc_test)).batch(32)


## Transfer Learning

En la fase de *transfer learning* se congela el encoder del modelo DistilBERT y se entrena únicamente la capa final de clasificación. De este modo, se reutilizan las representaciones semánticas generales aprendidas durante el preentrenamiento, adaptando el modelo al problema específico con un número reducido de parámetros entrenables.

Esta estrategia permite un entrenamiento rápido y estable, reduciendo el riesgo de sobreajuste y proporcionando una base sólida antes de realizar ajustes más profundos en el modelo.

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2, use_safetensors=False)

# Congelar capas entrenadas (transfer learning)
model.distilbert.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])

history_head = model.fit(train_ds, validation_data=val_ds, epochs=5, verbose=0)

In [24]:
logits_val = model.predict(val_ds).logits
y_pred_val = np.argmax(logits_val, axis=1)
mcc = matthews_corrcoef(y_val, y_pred_val)
print("MCC transfer learning:", mcc)

45/45 [==============================] - 9s 156ms/step
MCC transfer learning: 0.8692983222653029


## Fine-Tuning

Una vez entrenada la cabeza de clasificación, se procede a una fase de *fine-tuning* parcial del modelo. En esta etapa se descongelan las capas superiores del encoder, permitiendo que el modelo ajuste sus representaciones internas al dominio concreto de los mensajes de spam.

El fine-tuning se realiza utilizando una tasa de aprendizaje reducida, con el objetivo de realizar ajustes graduales sin destruir el conocimiento previo aprendido. Esta fase suele producir una mejora significativa en el rendimiento final del modelo.

In [25]:
model.distilbert.trainable = True

# congelamos todas menos las últimas 2
for layer in model.distilbert.transformer.layer[:-2]:
    layer.trainable = False
for layer in model.distilbert.transformer.layer[-2:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

history_ft = model.fit(train_ds, validation_data=val_ds, epochs=5, verbose=0)

In [27]:
logits_val = model.predict(val_ds).logits
y_pred_val = np.argmax(logits_val, axis=1)
mcc = matthews_corrcoef(y_val, y_pred_val)
print("MCC (fine-tuned):", mcc)

45/45 [==============================] - 7s 156ms/step
MCC (fine-tuned): 0.93885797199061


## Conclusiones del modelo importado

El modelo basado en DistilBERT logra una mejora respecto al modelo NLP básico, alcanzando valores de MCC significativamente superiores tras el proceso de fine-tuning.

Estos resultados muestran la capacidad de los modelos preentrenados para capturar relaciones semánticas complejas y generalizar mejor a ejemplos no vistos. A pesar de su mayor coste computacional, el enfoque basado en Transformers demuestra ser especialmente eficaz para tareas de clasificación de texto.

## Conclusiones generales

En este proyecto se han explorado dos enfoques complementarios para la clasificación SPAM/NO SPAM: un modelo NLP clásico basado en TF-IDF y regresión logística, y un modelo avanzado basado en un Transformer preentrenado.

El modelo básico ofrece una solución eficiente y fácil de interpretar, mientras que el modelo importado mediante transfer learning proporciona un rendimiento superior gracias a representaciones contextualizadas del lenguaje. La comparación entre ambos enfoques permite apreciar las ventajas y limitaciones de cada uno, así como la evolución natural desde técnicas clásicas hacia modelos modernos en NLP.

En conjunto, el proyecto demuestra la importancia de combinar fundamentos teóricos sólidos con modelos avanzados para resolver problemas reales de procesamiento del lenguaje natural.

## Submit your predictions in a `submission.csv` file for scoring on the [leaderboard](https://www.kaggle.com/competitions/u-tad-spam-not-spam-2025-edition/leaderboard)
To submit your notebook click on **Submit to competition** and then **Submit**.

In [29]:
logits_test = model.predict(test_ds).logits
y_pred_test = np.argmax(logits_test, axis=1).astype(int)

148/148 [==============================] - 23s 157ms/step


In [30]:
# do not modify this code
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv',index=False)

In [31]:
submission.head()

,row_id,spam_label
0,7097,0
1,7098,1
2,7099,0
3,7100,0
4,7101,0
